#### Names of people in the group

Please write the names of the people in your group in the next cell.

Jacob Ørmen Clements

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame that I have altered to work with my code
def load_df(table_name: "name of the table to load") -> DataFrame:
    # Match the path with the one used in the save_df function
    path = f"dbfs:/user/hive/warehouse/{table_name}.parquet"
    return spark.read.parquet(path)

users_df = load_df("users")
comments_df = load_df("comments")
posts_df = load_df("posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

## Note:
I will try to comment most of the lines of code that are related to Spark framework (operations on the DataFrame)to show comprehension of its features and functions

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    df.createOrReplaceTempView("df") # Registers the DataFrame as a SQL view in Spark's memory which allows SQL queries directly on the DataFrame
    result = spark.sql(query) # Executes the SQL query which becomes a Dataframed of the transformed data
    first_row = result.first()  # Extracts the first row from the result
    return first_row[0] if first_row is not None else None #Returns the value from the first column of the first row or None if non-exsistent

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    # We are working with two DataFrames this time, so we register both in the SQL context
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    result = spark.sql(query) # We run the SQL query which could potentially employ both DataFrames
    first_row = result.first() #Same as last function
    return first_row[0] if first_row is not None else None #Same as last time


In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
#SQL logic explains itself

q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

q2 = "SELECT COUNT(*) FROM df"

q3 = """
SELECT OwnerUserId AS user_id, COUNT(*) AS answers_count
FROM df
WHERE PostTypeId = 2
GROUP BY OwnerUserId
ORDER BY answers_count DESC
"""

q4 = "SELECT COUNT(*) AS total_questions FROM df WHERE PostTypeId = 1"

q5 = """
SELECT u.DisplayName, COUNT(*) AS comments_count
FROM df1 u
JOIN df2 c ON u.Id = c.UserId
GROUP BY u.DisplayName
ORDER BY comments_count DESC
"""



#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 5.566s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

# Answers:
### Difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD):
RDDs are the foundational data structures of Spark designed for low-level operations which leads them to being robust. The robustness comes from RDDs offering granular control over data distribution and fault tolerance. DataFrames, are at a higher abstraction level, where you can think of them as organized data in named columns (like relational databases). This feature makes DataFrames easy to use for those familiar with the SQL commands or other uses on tabular forms. For Datasets we have the best of both words as they combine the best features of RDDs and DataFrames. They do this by ensuring type safety and familiar programming models of RDDs with the efficiency and simplicity of the DataFrame.

### When to use RDDs instead of DataFrames:
RDDs excel in the low-level programming, where meticulous control over the data distribution is imperative. Some prevalent situations for using RDDs is when optimizing for specific computations or dealing with unstructured data in the form of logs or streams. Moreover, you might think of some algortihms you wnat to run on your data which does not exist for the alternative data structures. Consequently, using RDDs can be useful in this sense as well.

### Main benefit of using DataSets instead of DataFrames:
The main benefit of Datasets over DataFrames is it's compile-time type safety. Owing to this feature, Datasets help catch potential errors during compilation rather than at runtime. Therefore, we can save precious debugging time whilst also ensuring code reliability by employing this data structure. Additionally, Datasets have the same capabilities of DataFrames, so we don't lose any of "goodness" related to DataFrames.